In [49]:
!nvidia-smi

Fri May 14 20:38:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K40m          Off  | 00000000:81:00.0 Off |                    0 |
| N/A   58C    P0   133W / 235W |   3648MiB / 11441MiB |     93%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%cd /home/mtech1/19CS60R28/susmit/DocRed_hongwang600/DocRed
import sys
import nltk
import numpy as np
import os
import json
import copy

from nltk.tokenize import WordPunctTokenizer
from pytorch_transformers import *
from models.bert import Bert
import seaborn as sns
%cd ..

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

/home/mtech1/19CS60R28/susmit/DocRed_hongwang600/DocRed
/home/mtech1/19CS60R28/susmit/DocRed_hongwang600


# Data

In [3]:
in_path='data'
out_path='prepro_data'

In [4]:
train_annotated_file_name = os.path.join(in_path, 'train_annotated.json')
dev_file_name = os.path.join(in_path, 'dev.json')
test_file_name = os.path.join(in_path, 'test.json')

In [5]:
rel2id = json.load(open(os.path.join(out_path, 'rel2id.json'), "r"))
id2rel = {v:u for u,v in rel2id.items()}
json.dump(id2rel, open(os.path.join(out_path, 'id2rel.json'), "w"))

In [6]:
#these are as per Bert class 
SEP='[SEP]'
MASK = '[MASK]'
CLS = "[CLS]"
model_name='bert'
PRE_TRAINED_MODEL_NAME='bert-base-uncased'

bert = Bert(BertModel, PRE_TRAINED_MODEL_NAME)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


# Preprocessing

In [11]:
def logging(s, print_=True, log_=True):
    if print_:
        print(s)
    if log_:
        with open(os.path.join(os.path.join("log", model_name+'.txt')), 'a+') as f_log:
            f_log.write(str(s)+'\n')

In [127]:
#to do add union of evidences=> done
# for sent pad use 2

def preprocess(data_file_name, max_length = 512, is_training = True, suffix=''):
    ori_data=json.load(open(data_file_name))
    max_sent_count=0#maximum number of sentences in a doc across the dataset
    list_sent_ids=[]
    list_attention=[]#this stores attention of docs
    list_sent_mask=[]#this will be used in the batch multliplication for getting the embeddings of each sentence
    # (len(list_sent_ids),max_sent_count,max_length)
    
    labels=[]
    i=0
    for doc in ori_data:
        i=i+1
        sys.stdout.write("\r%d/%d docs"%(i,len(ori_data)))
        # this dict is used to take care of multiple relations with same head and tail
        head_tail_index={}
        max_sent_count=max(max_sent_count,len(doc['sents']))
        for label in doc['labels']:
            idx_list=[]
            head=doc['vertexSet'][label['h']]
            tail=doc['vertexSet'][label['t']]
            if (label['h'],label['t']) in head_tail_index:
                labels[head_tail_index[(label['h'],label['t'])]]+=label['evidence']
                continue
            else:
                head_tail_index[(label['h'],label['t'])]=len(list_sent_ids)
            for entity in head:
                if (entity['sent_id'],entity['pos'][0],'[unused0]') not in idx_list:
                    idx_list.append((entity['sent_id'],entity['pos'][0],'[unused0]'))
                if (entity['sent_id'],entity['pos'][1]+1,'[unused1]') not in idx_list:
                    idx_list.append((entity['sent_id'],entity['pos'][1]+1,'[unused1]'))
            for entity in tail:
                if (entity['sent_id'],entity['pos'][0],'[unused2]') not in idx_list:
                    idx_list.append((entity['sent_id'],entity['pos'][0],'[unused2]'))
                if (entity['sent_id'],entity['pos'][1]+1,'[unused3]') not in idx_list:
                    idx_list.append((entity['sent_id'],entity['pos'][1]+1,'[unused3]'))
            idx_list.sort(key=lambda tup:(tup[0],tup[1]),reverse=True)
            temp_doc=copy.deepcopy(doc)
            for loc in idx_list:
                temp_doc['sents'][loc[0]].insert(loc[1],loc[2])

            sent_combine=[]
            for sent in temp_doc['sents']:
                sent_combine=sent_combine+sent
            sent_ids,sent_attention_mask,sent_start_ids=bert.subword_tokenize_to_ids(sent_combine)
            list_sent_ids.append(sent_ids[0])
            list_attention.append(sent_attention_mask[0])
            labels.append(label['evidence'])
            
            
            sent_mask=[]
            l=1# we start from index 1 because we skip CLS token
            for sent in temp_doc['sents']:
                sent_mask.append([0]*max_length)
                j=l
#                 print(sent)
#                 print("\n")
                while(j<min(max_length-2,l+len(sent))):
                    sent_mask[-1][j]=1
                    j+=1
                l+=len(sent)
                if(l>=max_length-2):
                    break
            list_sent_mask.append(sent_mask)
            
    logging('')
    evi_labels = np.zeros((len(labels),max_sent_count),dtype = np.int64)
    for i in range(len(labels)):
        evi_labels[i][labels[i]]=1 #if evidence present then 1
    print("max_sent_cout",max_sent_count)
    for i in range(len(list_sent_mask)):
        # the label for pad sentence is 2
        evi_labels[i][len(list_sent_mask[i]):max_sent_count]=2
        # to pad sentences with arrays of 1s
        list_sent_mask[i]=list_sent_mask[i]+[[1]*max_length]*(max_sent_count-len(list_sent_mask[i]))
    list_sent_ids=np.asarray(list_sent_ids,dtype=np.int64)
    list_attention=np.asarray(list_attention,dtype=np.int64)
    list_sent_mask=np.asarray(list_sent_mask,dtype=np.int64)
    
    logging("Started saving")
    
    logging("Number of instances: {}".format(list_sent_ids.shape[0]))
    np.save(os.path.join(out_path,suffix+'_sent_ids.npy'),list_sent_ids)
    np.save(os.path.join(out_path,suffix+'_sent_attention_mask.npy'),list_attention)
    np.save(os.path.join(out_path,suffix+'_sent_mask.npy'),list_sent_mask)
    np.save(os.path.join(out_path,suffix+'_evidence_labels.npy'),evi_labels)
    logging("completed saving\n")
    


In [128]:
preprocess(train_annotated_file_name, max_length = 512, is_training = False, suffix='train')
preprocess(dev_file_name, max_length = 512, is_training = False, suffix='dev')

3053/3053 docs
max_sent_cout 25
Started saving
Number of instances: 35615
completed saving

1000/1000 docs
max_sent_cout 21
Started saving
Number of instances: 11518
completed saving



### Data loading

In [11]:
import torch
# import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F


In [11]:
class Docred_dataset(Dataset):
    def __init__(self,sent_ids,sent_mask,evi_target,max_len):
        self.sent_ids=sent_ids
        self.sent_mask=sent_mask
        self.ev_target=evi_target
    def __len__(self):
        return evi_target.shape[0]
    def __getitem__(self,item):
        return {
            'input_ids':torch.tensor(sent_ids,dtype=torch.long).flatten(),
            'attention_masks':torch.tensor(sent_mask,dtype=torch.long).flatten(),
            'targets':torch.tensor(evi_target,dtype=torch.long)
        }

In [12]:
sent_ids=np.load(os.path.join(out_path,'train'+'_sent_ids.npy'))
sent_mask=np.load(os.path.join(out_path,'train'+'_sent_mask.npy'))
evi_target=np.load(os.path.join(out_path,'train'+'_evidence_labels.npy'))

In [13]:
def create_data_loader(max_len,batch_size):
    ds=Docred(sent_ids,sent_mask,evi_target,max_len=512)
    return DataLoader(ds,batch_size=batch_size,num_workers=4)
BATCH_SIZE=8
MAX_LEN=512
train_data_loader=create_data_loader(max_len=MAX_LEN,batch_size=BATCH_SIZE)

In [14]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['input_ids', 'attention_masks', 'targets'])

In [15]:
print(data['input_ids'].shape)
print(data['attention_masks'].shape)
print(data['targets'].shape)

torch.Size([8, 61440])
torch.Size([8, 61440])
torch.Size([8, 120, 12])


### Evidence calssification using Bert and Hugging face

In [16]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [17]:
# sent_mask = (b,k,t)
# k=max sent length
# output =(b,t,h)
# torch.bmm
# (b,k,t)*(b,t,h)/(len(sent))==(b,k,h)
# torch.sum(dim)  torch.sum(input, dim, keepdim=False, *, dtype=None) → Tensor


class EvidenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EvidenceClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME,return_dict=False)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask,sent_mask):
        _, pooled_output = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        output = self.drop(pooled_output)
#         #BMM
#         2
#             10->3sent
#             5->2sent
#             sent_mask=(2,3,10)
#             length of each sent mask->10
#             1st doc->1st sent(0,3)
#             each sent_mask=[1,1,1,1,0,0,0,...]
#             2nd doc ->(pad sent) use all ones 
#             2nd doc->3rd sent->[1,1,1,...](to divide zero issue)
#             bert_output->(2,10,768)
#             sent_mask*bert_output(BMM)
#             output=(2,3,768)=(2,3,10)*(2,10,768)
#             output(2,3,3)=Linear(output,3)
        return self.out(output)

In [ ]:
#         loss(input(output of the model),target)
            #no_evidence_sent=>lablel=0
            #evidence_sent=>label=1
            #evidence_pad=>lebel=2
#             loss(ignore_index=2) 
#             NLLLoss
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html?highlight=nllloss#torch.nn.NLLLoss

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [57]:
input_ids = torch.tensor(tokenizer.encode("[CLS] Hello my dog is cute")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_state, pooler_output = outputs

In [58]:
print(last_hidden_state.shape)
print(pooler_output.shape)

torch.Size([1, 6, 768])
torch.Size([1, 768])


In [62]:
print(last_hidden_state[0][0])

tensor([-7.0670e-02, -1.3634e-01, -2.0471e-01,  1.3662e-01, -5.4963e-02,
         1.3121e-01, -1.6027e-01, -1.4354e-01, -3.3725e-01, -2.0080e-01,
        -3.8223e-02,  2.4945e-01, -3.3377e-01, -1.7753e-01, -4.1003e-01,
         7.3531e-01, -3.7430e-02, -7.1812e-02, -4.7473e-01, -5.2340e-02,
         3.7171e-01, -1.2110e-01,  1.4048e-01, -6.6024e-02,  4.0694e-01,
        -1.5038e-01,  6.9376e-01,  3.7555e-03, -3.3062e-01, -8.2766e-02,
        -6.4227e-02, -2.2916e-01,  2.4486e-01,  1.0927e-01,  2.3430e-01,
        -3.7256e-01, -3.8063e-02, -2.4527e-01, -3.1919e-01,  5.5296e-01,
        -9.9116e-02, -5.8969e-01, -7.5447e-02,  2.0883e-01, -2.2657e-01,
        -1.2772e-01, -1.1553e+00,  1.5863e-01,  2.9359e-01, -3.7036e-01,
        -4.3070e-01,  9.0894e-02,  6.8570e-02,  2.1841e-01,  2.0635e-02,
         1.8272e-01,  1.3496e-01, -1.7807e-01,  8.3814e-02,  1.2017e-01,
         7.9420e-02, -3.5725e-01,  6.3858e-03, -8.3377e-02, -1.4762e-01,
         1.3408e-01,  6.4399e-02,  6.1005e-02,  4.5